# Instituto Tecnológico de Costa Rica (ITCR)
# Ciencia de Datos

## Balanceo de datos (ejemplo)

-Profesora: María Auxiliadora Mora

## Introducción

El concepto **datos desbalanceados** se refiere a un problema relacionado con actividades de clasificación en donde las clases no están representadas por igual en el conjunto de datos.

**La mayoría de los conjuntos de datos utilizados en procesos de clasificación están desbalanceados por la naturaleza del fenómeno que representan**. Por ejemplo, en el caso de clasificación de imágenes de pacientes con cáncer de piel, se espera que la mayor parte de los exámenes realizados a pacientes va a pertencer a la clase "No cáncer".

Algunas recomendaciones:
- Para medir la precisión de modelos generados con datos desbalanceados la métrica del accuracy no es recomendada, es mejor utilizar la matriz de confusión y las métricas precision, recall y F1. Adicionalmente, se podrían utilizar las métricas Cohen’s kappa y la curva ROC.

### Ejemplo

Se va a utilizar un conjunto de datos con clases balanceadas y uno que contiene clases desbalancedas para comparar resultados de algoritmos y hacer evidente el impacto en los resultados. 

**Objetivo de la clasificación**: El principal objetivo de proceso de clasificación es el mismo en ambos casos, predecir si las recetas y lista de ingredientes son indias o no. El atributo is_indian es 1 si la receta corresponde a comida india y 0 si no. 

**Conjunto de datos**

Se va a utilizar dos conjuntos de datos, uno balanceado y el otro no: 
- recipes.csv: datos desbalanceados
- recipes-indian.csv: datos balanceados

Ambos conjuntos de datos contienen el origen de la receta en la columna (cuisine), la lista de ingredientes (ingredient_list) y la clase a la que pertenecen que corresponde a 1 si es comida india y 0 si no (is_indian).



In [1]:
#!pip install -U imbalanced-learn

In [13]:
# Libraries
import pandas as pd

# evaluate models applied to unbalanced data
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# imbalanced-learn to to preprocess the unbalanced data
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import ADASYN, SMOTE

In [14]:
# Load balanced datasets
ingredients_balanced_dataset = pd.read_csv('../../Data/recipes-indian.csv')

ingredients_balanced_dataset.head()

,cuisine,id,ingredient_list,is_indian
0,indian,23348,"minced ginger, garlic, oil, coriander powder, ...",1
1,indian,18869,"chicken, chicken breasts",1
2,indian,36405,"flour, rose essence, frying oil, powdered milk...",1
3,indian,11494,"soda, ghee, sugar, khoa, maida flour, milk, oil",1
4,indian,32675,"tumeric, garam masala, salt, chicken, curry le...",1


In [15]:
# Records per class

ingredients_balanced_dataset.is_indian.value_counts()

1    3000
0    3000
Name: is_indian, dtype: int64

In [16]:
# Load un-balanced datasets
ingredients_unbalanced_dataset = pd.read_csv('../../Data/recipes.csv')
ingredients_unbalanced_dataset['is_indian'] = (ingredients_unbalanced_dataset.cuisine == "indian").astype(int)


ingredients_unbalanced_dataset.head()

,cuisine,id,ingredient_list,is_indian
0,greek,10259,"romaine lettuce, black olives, grape tomatoes,...",0
1,southern_us,25693,"plain flour, ground pepper, salt, tomatoes, gr...",0
2,filipino,20130,"eggs, pepper, salt, mayonaise, cooking oil, gr...",0
3,indian,22213,"water, vegetable oil, wheat, salt",1
4,indian,13162,"black pepper, shallots, cornflour, cayenne pep...",1


In [17]:
# Records per class

ingredients_unbalanced_dataset.is_indian.value_counts()

0    36771
1     3003
Name: is_indian, dtype: int64

## Ejemplo de Procesamiento de Lengiaje Natural (NLP) 

Objetivo clasificar los contenidos del texto para predecir si corresponden a comidia India o no. Primero utilizando los datos balanceados y luego los desbalanceados para comprarar los resultados.

Note que en NLP: que la extracción de características consiste en vectorizar los tokens que forman los textos de los ingredientes.


### Evaluación de modelos de clasificación:
Para evaluar los modelos resultantes se utilizará una **Matriz de Confusión**:
La Matriz de Confusión es una métricas muy sencilla de entender y visualizar para calcular la **precisión y exactitud del modelo**. Es una de las métricas más utilizada para evaluar modelos en actividades de clasificación con dos o más clases

![](../imagenes/Matriz_confusion.jpg)

Imagen de https://consultorjava.com/blog/matriz-de-confusion-efectividad-del-modelo-de-clasificacion/

In [18]:
# utility functions

def classify_data(X_train, X_test, y_train, y_test):
    """
    Classify the recipies text and display a confusion matrix
    Parameters:
      X_train: training data.  Matrix with data vectorized unsing Tf-idf
      X_test: test data.
      y_train, y_test: target (the class, 1 = Indian recipe and 0 = no)
    """
    # Build a classifier and train it (Support Vector Machine)
    clf = LinearSVC()
    clf.fit(X_train, y_train)

    # Test our classifier and build a confusion matrix
    y_true = y_test
    y_pred = clf.predict(X_test)
    matrix = confusion_matrix(y_true, y_pred)

    return matrix

### Extracción de características en NLP

**feature_extraction.text.TfidfVectorizer** convierte una colección de documentos en una matriz de características TF-IDF.

In [19]:
# Results using the balanced dataset

# Vectorize the text field using Term Frequency–Inverse Document Frequency (TF-IDF) 
vectorizer = TfidfVectorizer()

# Apply the vectorizer instance to text.
balanced_matrix = vectorizer.fit_transform(ingredients_balanced_dataset.ingredient_list)

# The features are the balanced_matrix of tf-idf values
X = balanced_matrix
y = ingredients_balanced_dataset.is_indian

y.value_counts()

1    3000
0    3000
Name: is_indian, dtype: int64

In [20]:
# Explore vectorizer contents
print(X.toarray())

# The representation is not empty
print("Máximo de la fila 0",max(X.toarray()[0,:]))


[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Máximo de la fila 0 0.34941044709979274


In [21]:
print(vectorizer.vocabulary_)

{'minced': 1019, 'ginger': 670, 'garlic': 659, 'oil': 1100, 'coriander': 401, 'powder': 1264, 'chickpeas': 317, 'onions': 1108, 'chopped': 341, 'tomatoes': 1669, 'salt': 1406, 'lemon': 905, 'juice': 831, 'fenugreek': 569, 'leaves': 899, 'chili': 325, 'cumin': 460, 'seed': 1441, 'ground': 723, 'chicken': 314, 'breasts': 189, 'flour': 612, 'rose': 1375, 'essence': 551, 'frying': 642, 'powdered': 1265, 'milk': 1016, 'ghee': 667, 'sugar': 1592, 'baking': 86, 'soda': 1508, 'khoa': 862, 'maida': 956, 'tumeric': 1689, 'garam': 654, 'masala': 985, 'curry': 470, 'water': 1755, 'cinnamon': 353, 'sticks': 1568, 'fresh': 629, 'spinach': 1536, 'crushed': 446, 'red': 1334, 'pepper': 1188, 'flakes': 597, 'mustard': 1056, 'seeds': 1443, 'tomato': 1668, 'sauce': 1419, 'wine': 1780, 'vinegar': 1743, 'cilantro': 352, 'fat': 563, 'free': 624, 'less': 910, 'sodium': 1509, 'broth': 199, 'pork': 1252, 'tenderloin': 1635, 'diced': 492, 'canola': 247, 'brown': 200, 'rice': 1346, 'kaffir': 837, 'lime': 916, 'wh

### Clasificación con datos balanceados

In [26]:
# Results using balanced dataset
# Split into test and train data 
X_train, X_test, y_train, y_test = train_test_split(X, y)

# data classification.
matrix = classify_data(X_train, X_test, y_train, y_test )

# Visualize the confusion matrix
label_names = pd.Series(['not indian', 'indian'])
pd.DataFrame(matrix,
         columns='Predicted ' + label_names,
         index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)



,Predicted not indian,Predicted indian
Is not indian,0.992048,0.007952
Is indian,0.188482,0.811518


### Clasificación con datos desbalanceados

In [30]:
# Results using unbalanced dataset
# Create a vectorizer and train it
vectorizer = TfidfVectorizer()
unbalanced_matrix = vectorizer.fit_transform(ingredients_unbalanced_dataset.ingredient_list)

# Features are our matrix of tf-idf values
# labels are whether each recipe is Indian or not
X = unbalanced_matrix
y = ingredients_unbalanced_dataset.is_indian

# How many are Indian?
y.value_counts()

0    36771
1     3003
Name: is_indian, dtype: int64

In [32]:
# Split into test and train data 
X_train, X_test, y_train, y_test = train_test_split(X, y)

# data classification.
matrix = classify_data(X_train, X_test, y_train, y_test)

# Visualize the confusion matrix
pd.DataFrame(matrix,
         columns='Predicted ' + label_names,
         index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted not indian,Predicted indian
Is not indian,0.991849,0.008151
Is indian,0.165545,0.834455


### Resultados

Cuando se realizó la clasificación de los datos balanceados el resultado fue muy bueno (más del 90% de éxito en las predicciones). Con datos desbalanceados el resultado baja a larededor del 80%.  

Una forma fácil de explicar el resultado es que **cuando se trata de una decisión arriesgada, siempre es más seguro adivinar "no indio" porque hay más probabilidad de estar en lo correcto**. De hecho, si siempre supusiéramos que no es indio, pase lo que pase, el resultado del modelo sería muy bueno.

## Posibles soluciones 

**1) Sub-muestreo o subsampling**: Elimina las muestras de la clase con mayor número de muestras que sobrepasan la cantidad de muestras de la otra clase.

Se utilizará la técnica de **submuestreo** para disminuir las recetas no indias aleatoriamente de forma que  coincidan con la cantidad de recetas indias. Este proceso se debe realizar solo con los datos de entrenamiento.

In [34]:
# Under samplig
resampler = RandomUnderSampler()

# Resample X and y so there are equal numbers of each y
X_train_resampled, y_train_resampled = resampler.fit_resample(X_train, y_train)

# count samples per class.
y_train_resampled.value_counts()

0    2260
1    2260
Name: is_indian, dtype: int64

In [36]:
# data classification.
matrix = classify_data(X_train_resampled, X_test, y_train_resampled , y_test)

# Visualize the confusion matrix
pd.DataFrame(matrix,
         columns='Predicted ' + label_names,
         index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted not indian,Predicted indian
Is not indian,0.969677,0.030323
Is indian,0.063257,0.936743


**2) Sobre-muestreo u oversampling**: Consiste en generar las muestras faltantes para el conjunto de datos, repitiendo las muestras de la clase con menos muestras tantas veces sea necesario y de forma
aleatoria.

In [37]:
# Over sampling.
resampler = RandomOverSampler()

# Resample X and y so there are equal numbers of each y
X_train_resampled, y_train_resampled = resampler.fit_resample(X_train, y_train)

# count samples per class.
y_train_resampled.value_counts()

1    27570
0    27570
Name: is_indian, dtype: int64

In [38]:
# data classification.
matrix = classify_data(X_train_resampled, X_test, y_train_resampled , y_test)

# Visualize the confusion matrix
pd.DataFrame(matrix,
         columns='Predicted ' + label_names,
         index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted not indian,Predicted indian
Is not indian,0.969460,0.030540
Is indian,0.059219,0.940781


**3) Generación de nuevas muestras usando aprendizaje generativo**: El aprendizaje generativo se enfoca en aprender la distribución de los datos.

Además del muestreo aleatorio con reemplazo, la biblioteca imbalanced-learn implementa dos métodos populares para sobremuestrear las clases minoritarias: la técnica de **Sobremuestreo de Minorías Sintéticas (SMOTE)** y el método de **Muestreo Sintético Adaptativo (ADASYN)**. 

**3.1) la técnica de Sobremuestreo de Minorías Sintéticas (SMOTE):**

SMOTE es una técnica estadística de sobremuestreo de minorías sintéticas para aumentar el número de muestras de un conjunto de datos de forma equilibrada. **El algoritmo  genera nuevas instancias a partir de casos minoritarios existentes que se proporcionan como entrada**. 

Las instancias nuevas no son copias de los casos minoritarios existentes. En su lugar, **el algoritmo toma muestras del espacio de características de cada clase de destino y de sus vecinos más próximos**. Luego **genera nuevos ejemplos que combinan las características del caso que nos ocupa con características de sus vecinos**. Este enfoque aumenta las características disponibles para cada clase y hace que las muestras sean más generales.

SMOTE toma todo el conjunto de datos como una entrada, pero **solo aumenta el porcentaje de los casos minoritarios**. (Chawla et al., 2002)

In [39]:
# Advanced oversampling methods
# SMOTE
resampler = SMOTE()

# Resample X and y so there are equal numbers of each y
X_train_resampled, y_train_resampled = resampler.fit_resample(X_train, y_train)

# count samples per class.
y_train_resampled.value_counts()

1    27570
0    27570
Name: is_indian, dtype: int64

In [40]:
# data classification.
matrix = classify_data(X_train_resampled, X_test, y_train_resampled , y_test)

# Visualize the confusion matrix
pd.DataFrame(matrix,
         columns='Predicted ' + label_names,
         index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted not indian,Predicted indian
Is not indian,0.977732,0.022268
Is indian,0.100894,0.899106


**3.2.) Método de Muestreo Sintético Adaptativo (ADASYN)**

ADASYN es una extensión de SMOTE que busca darle mayor peso a los datos de la clase minoritaria que son difíciles de aprender (He et al., 2008).


In [41]:
# Advanced oversampling methods
# ADASYN
resampler = ADASYN()

# Resample X and y so there are equal numbers of each y
X_train_resampled, y_train_resampled = resampler.fit_resample(X_train, y_train)

# count samples per class.
y_train_resampled.value_counts()

1    27744
0    27610
Name: is_indian, dtype: int64

In [20]:
# data classification.
matrix = classify_data(X_train_resampled, X_test, y_train_resampled , y_test)

# Visualize the confusion matrix
pd.DataFrame(matrix,
         columns='Predicted ' + label_names,
         index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted not indian,Predicted indian
Is not indian,0.970540,0.029460
Is indian,0.089791,0.910209


## Conclusiones

- La división desigual de muestras pertenecientes a las clases objetivo puede causar un rendimiento subóptimo de los algoritmos de clasificación. 

- Se presentan tres diferentes enfoques para resolver el problema: submuestreo, sobremuestreo y generación de nuevas muestras usando aprendizaje generativo. Estos enfoques mejoran el rendimiento de los modelos en comparación con trabajar directamente con el conjunto de datos desequilibrado.


### Referencias

[1] Classification problems with imbalanced inputs. Recuperado de https://colab.research.google.com/github/littlecolumns/ds4j-notebooks/blob/master/classification/notebooks/Correcting%20for%20imbalanced%20datasets.ipynb#scrollTo=56BufhD27c-V

[2] imbalanced-learn.org. imbalanced-learn documentation. Recuperado de https://imbalanced-learn.org/stable/

[3] Chawla,N., Bowyer, K., Hall, L., and Kegelmeyer, P.(2002). Smote: synthetic minority over-sampling technique. Journal of artificial intelligence research, 16:321–357

[4] He, H., Bai, Y., Garcia, E. and Li, S. (2008). Adasyn: adaptive synthetic sampling approach for imbalanced learning. In 2008 IEEE International Joint Conference on Neural Networks (IEEE World Congress on Computational Intelligence), 1322–1328. IEEE.
